# Purpose

To download the metasploit JSON file and add a column to the CVE processed JSON file indicating for which CVEs there is a known metasploit exploit

In [64]:
import urllib.request
import json
import re
import pandas as pd
import numpy as np
import datetime
import dateutil

In [2]:
FILE_LOC = 'https://raw.githubusercontent.com/rapid7/metasploit-framework/master/db/modules_metadata_base.json'
METASPLOIT_RAW = '../../data/raw/metasploit.json'
CVE_PROCESSED = '../../data/processed/cves.json'
CVE_METASPLOIT_PROCESSED = '../../data/processed/cves_metasploit.json'

In [20]:
response = urllib.request.urlopen(FILE_LOC)
raw_file =  response.read()

with open(METASPLOIT_RAW, 'wb') as fout:
    fout.write(raw_file)
print("wrote metasploit.json")

wrote metasploit.json


In [3]:
# read metasploit file
with open(METASPLOIT_RAW, 'r') as fin:
    metasploit = json.loads(fin.read())

In [5]:
# read cve file as pandas
cves_df = pd.read_json(CVE_PROCESSED)
    
# add metasploit column set to 0
cves_df['metasploit'] = 0
cves_df['metasploit_date'] = ''

cves_df.head()

,access,authentication,availability,complexity,confidentiality,date,id,integrity,metasploit,metasploit_date
0,NETWORK,NONE,PARTIAL,LOW,NONE,1999-12-30,CVE-1999-0001,NONE,0,
1,NETWORK,NONE,COMPLETE,LOW,COMPLETE,1998-10-12,CVE-1999-0002,COMPLETE,0,
2,NETWORK,NONE,COMPLETE,LOW,COMPLETE,1998-04-01,CVE-1999-0003,COMPLETE,0,
3,NETWORK,NONE,PARTIAL,LOW,NONE,1997-12-16,CVE-1999-0004,NONE,0,
4,NETWORK,NONE,COMPLETE,LOW,COMPLETE,1998-07-20,CVE-1999-0005,COMPLETE,0,


In [18]:
cve_regx = re.compile(r'CVE-\d{4}-\d{4,7}')
total = len(metasploit)
count = 0
for key, item in metasploit.items():
    for reference in item['references']:
        match = cve_regx.match(reference)
        if match:
            matched_cve = match.group()
            matched = cves_df.loc[cves_df['id']==matched_cve, ['metasploit', 'metasploit_date']] = 1, item['disclosure_date']
            
    count += 1
    print("\r{0:.2f} Complete...".format(count*100/total), end='')

100.00 Complete...

In [19]:
cves_df.head()

,access,authentication,availability,complexity,confidentiality,date,id,integrity,metasploit,metasploit_date
0,NETWORK,NONE,PARTIAL,LOW,NONE,1999-12-30,CVE-1999-0001,NONE,0,
1,NETWORK,NONE,COMPLETE,LOW,COMPLETE,1998-10-12,CVE-1999-0002,COMPLETE,0,
2,NETWORK,NONE,COMPLETE,LOW,COMPLETE,1998-04-01,CVE-1999-0003,COMPLETE,0,
3,NETWORK,NONE,PARTIAL,LOW,NONE,1997-12-16,CVE-1999-0004,NONE,0,
4,NETWORK,NONE,COMPLETE,LOW,COMPLETE,1998-07-20,CVE-1999-0005,COMPLETE,0,


In [20]:
cves_df[cves_df['metasploit_date']!='']

,access,authentication,availability,complexity,confidentiality,date,id,integrity,metasploit,metasploit_date
101,NETWORK,NONE,PARTIAL,LOW,NONE,1996-02-08,CVE-1999-0103,NONE,1,1996-02-08
167,NETWORK,NONE,PARTIAL,LOW,PARTIAL,1997-01-01,CVE-1999-0170,PARTIAL,1,None
204,NETWORK,NONE,NONE,LOW,PARTIAL,1990-08-14,CVE-1999-0209,NONE,1,1994-12-12
251,NETWORK,NONE,PARTIAL,LOW,PARTIAL,1998-02-01,CVE-1999-0256,PARTIAL,1,1998-03-19
494,NETWORK,NONE,PARTIAL,LOW,PARTIAL,1998-03-01,CVE-1999-0502,PARTIAL,1,1999-01-01
496,NETWORK,NONE,PARTIAL,LOW,PARTIAL,1997-01-01,CVE-1999-0504,PARTIAL,1,1999-01-01
498,LOCAL,NONE,COMPLETE,LOW,COMPLETE,1998-10-01,CVE-1999-0506,COMPLETE,1,None
500,LOCAL,NONE,PARTIAL,LOW,PARTIAL,1998-06-01,CVE-1999-0508,PARTIAL,1,None
518,NETWORK,NONE,COMPLETE,LOW,COMPLETE,1997-07-01,CVE-1999-0526,COMPLETE,1,None
523,NETWORK,NONE,NONE,LOW,NONE,1997-07-01,CVE-1999-0532,NONE,1,None


In [22]:
# we apparently have some None metasploit_dates, lets turn those back to ''
cves_df.loc[cves_df['metasploit_date']==None, 'metasploit_date'] = ''

In [26]:
cves_df[cves_df['metasploit_date']==None]

,access,authentication,availability,complexity,confidentiality,date,id,integrity,metasploit,metasploit_date


In [31]:
cves_df[cves_df['date']==None]

,access,authentication,availability,complexity,confidentiality,date,id,integrity,metasploit,metasploit_date,report_delay


In [32]:
cves_df['date'].describe()

count                  102947
unique                   5196
top       2004-12-31 00:00:00
freq                     1098
first     1988-10-01 00:00:00
last      2018-07-18 00:00:00
Name: date, dtype: object

In [43]:
cves_df['date'] = pd.to_datetime(cves_df['date'])
cves_df['metasploit_date'] = pd.to_datetime(cves_df['metasploit_date'])
cves_df['report_delay'] = cves_df['metasploit_date'] - cves_df['date']

In [53]:
cves_df.head()

,access,authentication,availability,complexity,confidentiality,date,id,integrity,metasploit,metasploit_date,report_delay
0,NETWORK,NONE,PARTIAL,LOW,NONE,1999-12-30,CVE-1999-0001,NONE,0,NaT,NaT
1,NETWORK,NONE,COMPLETE,LOW,COMPLETE,1998-10-12,CVE-1999-0002,COMPLETE,0,NaT,NaT
2,NETWORK,NONE,COMPLETE,LOW,COMPLETE,1998-04-01,CVE-1999-0003,COMPLETE,0,NaT,NaT
3,NETWORK,NONE,PARTIAL,LOW,NONE,1997-12-16,CVE-1999-0004,NONE,0,NaT,NaT
4,NETWORK,NONE,COMPLETE,LOW,COMPLETE,1998-07-20,CVE-1999-0005,COMPLETE,0,NaT,NaT


In [58]:
cves_df[cves_df['metasploit_date'].notnull()]['report_delay'].describe()

count                        1530
mean           -66 days +03:28:00
std      399 days 08:00:22.937109
min          -3326 days +00:00:00
25%            -37 days +00:00:00
50%             -5 days +00:00:00
75%             -1 days +00:00:00
max            4465 days 00:00:00
Name: report_delay, dtype: object

In [65]:
cves_df[cves_df['report_delay']<datetime.timedelta(0)]

,access,authentication,availability,complexity,confidentiality,date,id,integrity,metasploit,metasploit_date,report_delay
960,NETWORK,NONE,COMPLETE,LOW,COMPLETE,1999-07-19,CVE-1999-1011,COMPLETE,1,1998-07-17,-367 days
1787,NETWORK,NONE,COMPLETE,LOW,COMPLETE,2000-04-24,CVE-2000-0248,COMPLETE,1,2000-04-04,-20 days
1861,NETWORK,NONE,COMPLETE,LOW,COMPLETE,2000-04-24,CVE-2000-0322,COMPLETE,1,2000-04-04,-20 days
2110,NETWORK,NONE,COMPLETE,LOW,COMPLETE,2000-07-07,CVE-2000-0573,COMPLETE,1,2000-06-22,-15 days
2447,NETWORK,NONE,COMPLETE,LOW,COMPLETE,2000-12-19,CVE-2000-0917,COMPLETE,1,2000-09-25,-85 days
2475,NETWORK,NONE,COMPLETE,LOW,COMPLETE,2000-12-19,CVE-2000-0945,COMPLETE,1,2000-10-26,-54 days
2618,NETWORK,NONE,COMPLETE,LOW,COMPLETE,2001-01-09,CVE-2000-1089,COMPLETE,1,2000-12-04,-36 days
2737,NETWORK,NONE,COMPLETE,LOW,COMPLETE,2002-08-12,CVE-2000-1209,COMPLETE,1,2000-05-30,-804 days
2928,LOCAL,NONE,NONE,LOW,PARTIAL,2001-05-03,CVE-2001-0152,NONE,1,2001-03-27,-37 days
2941,NETWORK,NONE,COMPLETE,HIGH,COMPLETE,2001-05-03,CVE-2001-0167,COMPLETE,1,2001-01-29,-94 days


In [68]:
cves_pos_df = cves_df[cves_df['report_delay']>=datetime.timedelta(0)]
cves_neg_df = cves_df[cves_df['report_delay']<datetime.timedelta(0)]

In [73]:
cves_neg_df['date'].count()

1245

In [74]:
cves_pos_df['date'].count()

285

In [27]:
out = cves_df.to_json(orient='records')
with open(CVE_METASPLOIT_PROCESSED, 'w') as fout:
    fout.write(out)
